In [1]:
import time

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

import config

In [2]:
driver = webdriver.Chrome()  # Mac用の記述
driver.implicitly_wait(10)

In [3]:
# SBIネオモバイルのログインページへアクセス
url_login = "https://trade.sbineomobile.co.jp/login"
driver.get(url_login)
time.sleep(3)  # ページに遷移する前に次の処理が実行されないようにするため

In [4]:
# ログインフォームの要素を取得する
username = driver.find_element(By.NAME, "username")
password = driver.find_element(By.NAME, "password")
login_btn = driver.find_element(By.ID, "neo-login-btn")

# 念のためテキストボックスの中身を空にする
username.clear()
password.clear()

# テキストボックスに値を入力する
username.send_keys(config.SBI_USERNAME)
password.send_keys(config.SBI_PASSWORD)

# ログインボタンをクリックする
login_btn.click()
time.sleep(1)

In [5]:
# SBIネオモバイルのポートフォリオ
url_portfolio = "https://trade.sbineomobile.co.jp/account/portfolio"
driver.get(url_portfolio)
time.sleep(3)

In [6]:
# 銘柄一覧の要素を取得する
div_element = driver.find_element(By.CLASS_NAME, "sp-main")

# 無限スクロールが終わるまでスクロールする
last_height = driver.execute_script("return arguments[0].scrollHeight", div_element)
while True:
    driver.execute_script(
        "arguments[0].scrollTo(0, arguments[0].scrollHeight);", div_element
    )
    time.sleep(2)
    new_height = driver.execute_script("return arguments[0].scrollHeight", div_element)
    if new_height == last_height:
        break
    last_height = new_height

In [7]:
# ページのhtmlを取得してパースする
html = driver.page_source.encode("utf-8")
parsed_html = BeautifulSoup(html, "html.parser")

# 全銘柄の現在値〜預り区分をデータフレームのリストにする
table = parsed_html.find_all("table")
list_df_tables = pd.read_html(str(table))

In [8]:
# リストのデータフレームを一つに結合する
df_all_stock = pd.DataFrame()
for x in list_df_tables:
    row = x.T[1:2]
    df_all_stock = pd.concat([df_all_stock, row], axis=0)

# カラム名を付け直す
df_all_stock.columns = ["現在値/前日比", "保有数量", "（うち売却注文中）", "評価損益率", "平均取得単価", "預り区分"]

# インデックスを振り直す
df_all_stock = df_all_stock.reset_index(drop=True)

In [11]:
# 証券コード、銘柄名を取得する
stock_code_list = []
ticker = parsed_html.find_all(class_="ticker")

# for x in ticker:
print(ticker)

[<p class="ticker">1434</p>, <p class="ticker">1518</p>, <p class="ticker">1775</p>, <p class="ticker">1820</p>, <p class="ticker">1835</p>, <p class="ticker">1925</p>, <p class="ticker">1928</p>, <p class="ticker">1951</p>, <p class="ticker">2124</p>, <p class="ticker">2127</p>, <p class="ticker">2169</p>, <p class="ticker">2353</p>, <p class="ticker">2790</p>, <p class="ticker">2809</p>, <p class="ticker">2914</p>, <p class="ticker">2927</p>, <p class="ticker">3003</p>, <p class="ticker">3101</p>, <p class="ticker">3156</p>, <p class="ticker">3179</p>, <p class="ticker">3186</p>, <p class="ticker">3248</p>, <p class="ticker">3409</p>, <p class="ticker">3632</p>, <p class="ticker">3817</p>, <p class="ticker">3847</p>, <p class="ticker">4008</p>, <p class="ticker">4042</p>, <p class="ticker">4044</p>, <p class="ticker">4064</p>, <p class="ticker">4182</p>, <p class="ticker">4361</p>, <p class="ticker">4385</p>, <p class="ticker">4452</p>, <p class="ticker">4502</p>, <p class="ticker">4